In this notebook, we test out hyperparameter tuning with Optuna. again. with GNBlocks and only 5 edges per node. set all config params fixed matching GNBLock_baseline except learning rate.

In [15]:
# ipython extension to autoreload imported modules so that any changes will be up to date before running code in this nb
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
from utils.jraph_training import train_and_evaluate_with_data, create_dataset
# from utils.jraph_models import MLPGraphNetwork
from utils.jraph_data import print_graph_fts
import ml_collections
import optuna 
from functools import partial
from datetime import datetime
import os 

In [17]:
# set up logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARNING)

### set up functions for optuna

In [18]:
CHECKPOINT_PATH = "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/experiments/tuning"

In [19]:
def objective(trial, datasets):
    """ Defines the objective function to be optimized over, aka the validation loss of a model.
    
        Args:
            trial: object which characterizes the current run 
            datasets: dictionary of data. we explicitly pass this in so that we don't have to waste runtime regenerating the same dataset over and over. 
    """
    # create config 
    config = ml_collections.ConfigDict()

    # Optimizer.
    config.optimizer = "adam"
    # config.optimizer = trial.suggest_categorical("optimizer", ["adam", "sgd"])
    config.learning_rate = trial.suggest_float('learning_rate', 
                                               0.99e-3, 1.01e-3, 
                                               log=True)
    # if config.optimizer == "sgd":
    #     config.momentum = trial.suggest_float('momentum', 0, 0.999) # upper bound is inclusive, and we want to exclude a momentum of 1 because that would yield no decay 

    # Data params that are used in training 
    config.output_steps = 4

    # Training hyperparameters.
    config.batch_size = 1 # variable currently not used
    config.epochs = 5
    config.log_every_epochs = 5
    config.eval_every_epochs = 5
    config.checkpoint_every_epochs = 10

    # GNN hyperparameters.
    config.model = 'MLPBlock'
    config.dropout_rate = 0.1
    config.skip_connections = False # This was throwing a broadcast error in add_graphs_tuples_nodes when this was set to True
    config.layer_norm = False # TODO perhaps we want to turn on later
    config.edge_features = (4, 8) # the last feature size will be the number of features that the graph predicts
    config.node_features = (32, 2)
    config.global_features = None

    # generate a workdir 
    # TODO: check if we actually care about referencing this in the future or if we can just create a temp dir 
    workdir=os.path.join(CHECKPOINT_PATH, "checkpoints", str(datetime.now()))

    # run training 
    state, train_metrics, eval_metrics_dict = train_and_evaluate_with_data(config=config, workdir=workdir, datasets=datasets)
    
    # retrieve and return val loss (MSE)
    print("eval_metrics_dict['val'].loss", eval_metrics_dict['val'].loss)
    print()
    return eval_metrics_dict['val'].loss.total




In [20]:
def get_data_config():
    config = ml_collections.ConfigDict()

    config.n_samples=100
    config.input_steps=1
    config.output_delay=8 # predict 24 hrs into the future 
    config.output_steps=4
    config.timestep_duration=3 # equivalent to 3 hours
    # note a 3 hour timestep resolution would be 5*24/3=40
    # if the time_resolution is 120, then a sampling frequency of 3 would achieve a 3 hour timestep 
    config.sample_buffer = -1 * (config.input_steps + config.output_delay + config.output_steps - 1) # negative buffer so that our sample input are continuous (i.e. the first sample would overlap a bit with consecutive samples) 
        # number of timesteps strictly between the end 
        # of one full sample and the start of the next sample
    config.time_resolution=120 # the number of 
                # raw data points generated per time unit, equivalent to the 
                # number of data points generated per 5 days in the simulation
    config.init_buffer_samples=100
    config.train_pct=0.7
    config.val_pct=0.2
    config.test_pct=0.1
    config.K=36
    config.F=8
    config.c=10
    config.b=10
    config.h=1
    config.seed=42
    config.normalize=True
    config.fully_connected_edges=False

    return config

In [21]:
def prepare_study(study_name):
    # generate dataset 
    dataset_config = get_data_config()
    datasets = create_dataset(dataset_config)
    print_graph_fts(datasets['train']['inputs'][0][0])

    # get the objective function that reuses the pre-generated datasets 
    objective_with_dataset = partial(objective, datasets=datasets)

    # run optimization study
    db_path = os.path.join(CHECKPOINT_PATH, study_name, "optuna_hparam_search.db")
    if not os.path.exists(os.path.join(CHECKPOINT_PATH, study_name)):
        os.makedirs(os.path.join(CHECKPOINT_PATH, study_name))

    study = optuna.create_study(
        study_name=study_name,
        storage=f'sqlite:///{db_path}', # generates a new db if it doesn't exist
        direction='minimize',
        pruner=optuna.pruners.MedianPruner(
            n_startup_trials=5, 
            n_warmup_steps=3
            ), 
        load_if_exists=True, 
    )
    # study.optimize(objective_with_dataset, 
    #                n_trials=25-len(study.trials), 
    #                n_jobs=1)
    
    return study, objective_with_dataset

### try hyperparameter tuning again with fewer params and lower learning rate options

In [22]:
# get study
study6, objective_with_dataset = prepare_study(study_name="hparam_study_6")

[I 2023-11-30 23:54:13,475] A new study created in RDB with name: hparam_study_6


Number of nodes: 36
Number of edges: 180
Node features shape: (36, 2)
Edge features shape: (180, 1)
Global features shape: (1, 1)


In [23]:
study6.optimize(objective_with_dataset, 
                n_trials=5-len(study6.trials), 
                n_jobs=1)

ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:54:20,061] Trial 0 finished with value: 22.606700897216797 and parameters: {'learning_rate': 0.001000177366474661}. Best is trial 0 with value: 22.6067008

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.6067, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:54:26,743] Trial 1 finished with value: 22.606460571289062 and parameters: {'learning_rate': 0.001005141750960963}. Best is trial 1 with value: 22.6064605

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.60646, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:54:33,679] Trial 2 finished with value: 22.607192993164062 and parameters: {'learning_rate': 0.001006591964736368}. Best is trial 1 with value: 22.6064605

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.607193, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:54:39,898] Trial 3 finished with value: 22.606624603271484 and parameters: {'learning_rate': 0.000997941584165578}. Best is trial 1 with value: 22.6064605

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.606625, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:54:46,378] Trial 4 finished with value: 22.60662269592285 and parameters: {'learning_rate': 0.0009980607383385516}. Best is trial 1 with value: 22.6064605

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.606623, dtype=float32), count=Array(20., dtype=float32))



In [27]:
study6.trials

[FrozenTrial(number=0, state=TrialState.COMPLETE, values=[22.606700897216797], datetime_start=datetime.datetime(2023, 11, 30, 23, 54, 13, 607591), datetime_complete=datetime.datetime(2023, 11, 30, 23, 54, 20, 44960), params={'learning_rate': 0.001000177366474661}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.00101, log=True, low=0.00099, step=None)}, trial_id=1, value=None),
 FrozenTrial(number=1, state=TrialState.COMPLETE, values=[22.606460571289062], datetime_start=datetime.datetime(2023, 11, 30, 23, 54, 20, 72128), datetime_complete=datetime.datetime(2023, 11, 30, 23, 54, 26, 731620), params={'learning_rate': 0.001005141750960963}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.00101, log=True, low=0.00099, step=None)}, trial_id=2, value=None),
 FrozenTrial(number=2, state=TrialState.COMPLETE, values=[22.607192993164062], datetime_start=datetime.datet

ok something is definitely wrong with the tuner. even when we pinned down all params except the learning rate which we restricted to (1 +- 0.1) e-3, the error was still 22 ????? how??? 

ok let's try visualizing this to see what in the hell is going wrong

In [24]:
fig = optuna.visualization.plot_intermediate_values(study6)
fig.show()

[W 2023-11-30 23:54:46,543] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [25]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study6)
fig.show()

In [26]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study6, params=['learning_rate', 'node_mlp_1_power'])
fig.show()

ValueError: Parameter node_mlp_1_power does not exist in your study.

ok literally why is this obsessed with the number 22.

In [ ]:
study6.optimize(objective_with_dataset, 
                n_trials=30-len(study6.trials), 
                n_jobs=1)

ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:45:48,250] Trial 5 finished with value: 20.664281845092773 and parameters: {'learning_rate': 0.007746910405710294, 'dropout_rate': 0.005763556253454017, '

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(20.664282, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:45:55,042] Trial 6 finished with value: 22.660564422607422 and parameters: {'learning_rate': 0.0043461527964268765, 'dropout_rate': 0.03081314767586012, '

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:46:01,617] Trial 7 finished with value: 22.658796310424805 and parameters: {'learning_rate': 0.007605965619586836, 'dropout_rate': 0.1150539479411566, 'ed

...

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.660564, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:48:10,972] Trial 25 finished with value: 22.649972915649414 and parameters: {'learning_rate': 0.0067550014708964495, 'dropout_rate': 0.01955952721295395, 

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(22.649973, dtype=float32), count=Array(20., dtype=float32))



ERROR:absl:Could not start profiling: Profile has already been started. Only one profile may be run at a time.
Traceback (most recent call last):
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/periodic_actions.py", line 327, in _start_session
    profiler.start(logdir=self._logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/clu/profiler.py", line 38, in start
    jax.profiler.start_trace(logdir)
  File "/Users/h.lu/Documents/_code/_research lorenz code/lorenzGNN/lorenzvenv/lib/python3.9/site-packages/jax/_src/profiler.py", line 111, in start_trace
    raise RuntimeError("Profile has already been started. "
RuntimeError: Profile has already been started. Only one profile may be run at a time.
[I 2023-11-30 23:48:17,631] Trial 26 finished with value: 20.023805618286133 and parameters: {'learning_rate': 0.0036401057439994864, 'dropout_rate': 0.035521297553533275,

eval_metrics_dict['val'].loss Metric.from_output.<locals>.FromOutput(total=Array(20.023806, dtype=float32), count=Array(20., dtype=float32))



In [ ]:
study6.trials

[FrozenTrial(number=0, state=TrialState.COMPLETE, values=[22.609474182128906], datetime_start=datetime.datetime(2023, 11, 30, 23, 50, 4, 99805), datetime_complete=datetime.datetime(2023, 11, 30, 23, 50, 10, 495863), params={'learning_rate': 0.0015869319524094753}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.008, log=True, low=0.0008, step=None)}, trial_id=1, value=None),
 FrozenTrial(number=1, state=TrialState.COMPLETE, values=[22.64401626586914], datetime_start=datetime.datetime(2023, 11, 30, 23, 50, 10, 525402), datetime_complete=datetime.datetime(2023, 11, 30, 23, 50, 16, 27179), params={'learning_rate': 0.005483155702544503}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.008, log=True, low=0.0008, step=None)}, trial_id=2, value=None),
 FrozenTrial(number=2, state=TrialState.COMPLETE, values=[22.628145217895508], datetime_start=datetime.datetime(202

In [ ]:
fig = optuna.visualization.plot_intermediate_values(study6)
fig.show()

[W 2023-11-30 23:33:28,281] You need to set up the pruning feature to utilize `plot_intermediate_values()`


In [ ]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study6, params=['learning_rate', 'dropout_rate'])
fig.show()

In [ ]:
# plot the estimated accuracy surface over hyperparameters:
fig = optuna.visualization.plot_contour(study6, params=['learning_rate', 'node_mlp_1_power'])
fig.show()

In [ ]:
print(study6.direction)

StudyDirection.MINIMIZE


In [ ]:
study6.trials

[FrozenTrial(number=0, state=TrialState.COMPLETE, values=[22.660564422607422], datetime_start=datetime.datetime(2023, 11, 30, 23, 29, 23, 273088), datetime_complete=datetime.datetime(2023, 11, 30, 23, 29, 32, 213207), params={'learning_rate': 0.0034667424009722694, 'dropout_rate': 0.1964864104266136, 'edge_mlp_1_power': 1, 'edge_mlp_2_power': 4, 'node_mlp_1_power': 5, 'node_mlp_2_power': 6}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'learning_rate': FloatDistribution(high=0.01, log=True, low=0.0001, step=None), 'dropout_rate': FloatDistribution(high=0.6, log=False, low=0.0, step=None), 'edge_mlp_1_power': IntDistribution(high=5, log=False, low=1, step=1), 'edge_mlp_2_power': IntDistribution(high=5, log=False, low=1, step=1), 'node_mlp_1_power': IntDistribution(high=8, log=False, low=1, step=1), 'node_mlp_2_power': IntDistribution(high=8, log=False, low=1, step=1)}, trial_id=1, value=None),
 FrozenTrial(number=1, state=TrialState.COMPLETE, values=[22.6605644

### literally why is this obsessed with the nmber 22